测试用的csv文件：

In [ ]:
'''
id1,id2,relation
1001,2001,support
1002,2001,refute
1003,2002,support
1002,2003,refute
1004,2001,support
1005,2004,refute
'''

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


目前写的是把relations视作doc间的relationship，也就是id1, id2说的是文档1和2。

但是实际Libo写的要求是relations描述的是claim。

我还没想明白怎么把claim映射到docs来做一系列的公式计算，是不是应该利用claims relation 计算出两篇文档的相似值来构造出doc relation（如果是支持则为正数，反对则为负数），然后再利用以下的代码进行迭代更新doc的信任score？

In [2]:
import pandas as pd
import numpy as np

def customized_pagerank(relations_df, max_iterations=100, alpha=0.85, tolerance=1e-6):
    """
    多加了两个argument：
      alpha: 阻尼因子
      tolerance: 收敛值

    返回：各文档的信任得分
    """
    # 提取所有的文档 ID
    document_ids = set(relations_df['id1']).union(set(relations_df['id2']))
    document_ids = sorted(document_ids)
    doc_index = {doc_id: idx for idx, doc_id in enumerate(document_ids)}  # {1001: 0, 1002: 1, 1003: 2, 2001: 3}
    index_doc = {idx: doc_id for doc_id, idx in doc_index.items()}     # {0: 1001, 1: 1002, 2: 1003, 3: 2001}
    N = len(document_ids)

    # 初始化信任度向量 s_d0
    s = np.full(N, 0.5)
    s0 = s.copy()

    trusted_docs = []  # 这里添加手动标注过的可信文档的ID列表（初始值是1，不确定的保留0.5）
    for doc_id in trusted_docs:
        idx = doc_index[doc_id]
        s[idx] = 1.0
        s0[idx] = 1.0

    # 正向和负向的 adjacency matrix
    W_plus = np.zeros((N, N))
    W_minus = np.zeros((N, N))

    for _, row in relations_df.iterrows():
        id1 = row['id1']
        id2 = row['id2']
        relation = row['relation']
        i = doc_index[id1]
        j = doc_index[id2]
        if relation == 'support': # 后期改成数值
            W_plus[i, j] += 1
        elif relation == 'refute': # 后期改成数值
            W_minus[i, j] += 1

    # 计算每个文档的正向和负向权重总和
    W_plus_sum = W_plus.sum(axis=0)
    W_minus_sum = W_minus.sum(axis=0)

    for iteration in range(max_iterations):
        s_prev = s.copy()
        P_pos = np.zeros(N)
        N_neg = np.zeros(N)

        # 正负向影响
        for d in range(N):
            if W_plus_sum[d] > 0:
                P_pos[d] = np.dot(W_plus[:, d], s_prev) / W_plus_sum[d]
            else:
                P_pos[d] = 0

            if W_minus_sum[d] > 0:
                N_neg[d] = np.dot(W_minus[:, d], s_prev) / W_minus_sum[d]
            else:
                N_neg[d] = 0

        # 净影响 + 映射
        I = P_pos - N_neg
        f_I = (I + 1) / 2
        s = (1 - alpha) * s0 + alpha * f_I

        # convergence
        delta = np.linalg.norm(s - s_prev, ord=1)
        if delta < tolerance:
            print(f'迭代在第 {iteration+1} 次时收敛，变化量 {delta}')
            break

    scores = {index_doc[idx]: s[idx] for idx in range(N)}
    return scores


In [3]:
relations_df = pd.read_csv('/content/drive/My Drive/546 Project/relations.csv')
scores = customized_pagerank(relations_df, max_iterations=100)
for doc_id, score in scores.items():
    print(f'文档 {doc_id} 的信任得分为 {score:.4f}')

迭代在第 2 次时收敛，变化量 0.0
文档 1001 的信任得分为 0.5000
文档 1002 的信任得分为 0.5000
文档 1003 的信任得分为 0.5000
文档 1004 的信任得分为 0.5000
文档 1005 的信任得分为 0.5000
文档 2001 的信任得分为 0.5000
文档 2002 的信任得分为 0.7125
文档 2003 的信任得分为 0.2875
文档 2004 的信任得分为 0.2875


其他问题：
如果两个文档与其他具有已知信任度的文档没有连接，那么由于初始信任度相同，且彼此的影响对等，迭代更新后，信任度可能没啥变化。